# CryoTEMPO-EOLIS products over Austfonna Basin 3

### Introduction
This example shows how to use Specklia to retrieve [CryoTEMPO-EOLIS Point](https://cryotempo-eolis.org/point-product/) and [Gridded Product](https://cryotempo-eolis.org/gridded-product/) data over [Austfonna Basin 3](https://www.swisseduc.ch/glaciers/svalbard/austfonna/index-en.html).

You can view the code below or [click here](https://colab.research.google.com/github/earthwave/specklia_demo_notebooks/blob/main/austfonna_basin_3.ipynb/) to run it yourself in Google Colab!

### Environment Setup
To run this notebook, you will need to make sure that the folllowing packages are installed in your python environment (all can be installed via pip/conda):
- matplotlib
- geopandas
- contextily
- ipykernel
- shapely
- specklia

If you are using the Google Colab environment, these packages will be installed in the next cell. Please note this step may take a few minutes.

In [ ]:
%pip install rasterio --no-binary rasterio
%pip install specklia
%pip install matplotlib
%pip install geopandas
%pip install contextily
%pip install shapely
%pip install python-dotenv

Now, let's import the packages and create a Specklia client to work with:

In [ ]:
# fix an issue that can sometimes occur with rasterio using the wrong version of proj
import os
import pyproj
os.environ['PROJ_LIB'] = pyproj.datadir.get_data_dir()

from datetime import datetime
import pprint
from time import perf_counter

import contextily as ctx
from dotenv import load_dotenv
import geopandas as gpd
from IPython.display import display
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
from pandas import Timedelta
import shapely
from specklia import Specklia

# load a demonstration API key from a .env file.
load_dotenv()

# Specify polygon colors to match another Earthwave product, https://cs2eo.org.
data_coverage_color = np.array([52, 211, 153])/255
geo_filter_colour = np.array([96, 165, 250])/255

# To run this code yourself, first generate your own key using https://specklia.earthwave.co.uk.
if 'DEMO_API_KEY' in os.environ:
    client = Specklia(os.environ['DEMO_API_KEY'])
else:
    user_api_key = input('Please generate your own key using https://specklia.earthwave.co.uk/ApiKeys and paste it here:')
    client = Specklia(user_api_key)

### Listing groups and datasets
The demonstration user has the same permissions as a freshly created Specklia user.
Let's list the groups they are a part of and the datasets they can access:

In [ ]:
display(client.list_groups())

In [ ]:
available_datasets = client.list_datasets()
display(available_datasets)

### Plotting dataset spatial coverage
There's a lot of information here. Let's plot the geospatial coverage of the Point and Gridded product datasets, using [Contextily](https://contextily.readthedocs.io/en/latest/) to add a satellite background map. Note that as of 30th June 2023, only products covering Svalbard have been loaded. We will be loading more in the second half of 2023.

All background satellite imagery tiles are provided by the [Esri World Imagery Service](https://www.esriuk.com/en-gb/content/products?esri-world-imagery-service) via Contextily.

In [ ]:
eolis_datasets = {}
for ds_name in ['CryoTEMPO-EOLIS Point Product', 'CryoTEMPO-EOLIS Gridded Product']:
    eolis_datasets[ds_name] = available_datasets[
        available_datasets['dataset_name'] == ds_name].iloc[0]

    print(f"{ds_name} contains data timestamped between \n"
        f"{eolis_datasets[ds_name]['min_timestamp']} "
        f"and {eolis_datasets[ds_name]['max_timestamp']}\n")

    print(f"{ds_name} has the following columns:")
    pprint.PrettyPrinter(indent=2, width=120).pprint(eolis_datasets[ds_name]['columns'])
    print("\n\n")

    desired_dataset_spatial_coverage = gpd.GeoDataFrame(
        geometry=[eolis_datasets[ds_name]['epsg4326_coverage']], crs=4326)
    
    # we create two plots, one for each hemisphere, to minimise distortion
    # and illustrate that multiple regions may be present within the same dataset
    for crs, hemisphere_name, hemisphere_bounding_box in [
            (3413, 'northern', (-180, 0, 180, 90)),
            (3031, 'southern', (-180, -90, 180, 0))
            ]:

        cropped_data = desired_dataset_spatial_coverage.clip(hemisphere_bounding_box)

        if len(cropped_data) > 0:
            transformed_cropped_data = cropped_data.to_crs(crs)
            ax = transformed_cropped_data.plot(
                figsize=(10, 10), alpha=0.5, color=data_coverage_color, edgecolor=data_coverage_color)

            ax.set_xlabel('x (m)')
            ax.set_ylabel('y (m)')
            ax.set_title(f"{ds_name} spatial coverage, {hemisphere_name} hemisphere (EPSG {crs})")
            # we calculate the zoom level dynamically so that we don't have to change this code
            # when new regions are added to the CryoTEMPO EOLIS products.
            ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, crs=crs, attribution=False,
                            zoom=ctx.tile._calculate_zoom(*cropped_data.total_bounds) - 1)

### Querying RGI polygons

In this example, we are interested in studying Austfonna Basin 3. Conveniently, Specklia contains all [RGI v7.0](https://www.glims.org/RGI/) glacier boundary definitions. Let's retrieve the RGI v7.0 boundary of Basin 3 from Specklia and take a look.

Firstly, we need to define a rough polygon covering the area to query the RGIv7 polygons from Specklia.

In [ ]:
austfonna_extent = shapely.from_wkt(
    'POLYGON ((19.27002 79.492646, 23.862305 79.075977, 28.630371 79.843346, 25.708008 80.441282, 19.27002 79.492646))')
austfonna_extent_gdf = gpd.GeoSeries(austfonna_extent, crs=4326)
# We plot in the WGS 84 / NSIDC Sea Ice Polar Stereographic North CRS (EPSG 3413).
ax = austfonna_extent_gdf.to_crs(3413).plot(
    figsize=(10, 10), alpha=0.5, color=geo_filter_colour, edgecolor=geo_filter_colour)

ax.set_xlabel('x (meters)')
ax.set_ylabel('y (meters)')
ax.set_title("Query extent for Austfonna")
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, crs=3413, attribution=False, zoom=8)

Now that we have defined a rough extent covering Austfonna, we can query the polygons from Specklia. Specklia will return all RGI Glaciers whose centroids fall within the query polygon.

In [ ]:
rgiv7_dataset = available_datasets[available_datasets['dataset_name'] == 'RGIv7 Glaciers'].iloc[0]

query_start_time = perf_counter()

rgi_data, rgi_data_sources = client.query_dataset(
    dataset_id=rgiv7_dataset['dataset_id'],
    epsg4326_polygon=austfonna_extent,
    min_datetime=rgiv7_dataset['min_timestamp'] - Timedelta(seconds=1),
    max_datetime=rgiv7_dataset['max_timestamp'] + Timedelta(seconds=1))

print(f'Query took {perf_counter()-query_start_time:.2f} seconds to complete.')
print(f'Austfonna Polygon Query complete, {len(rgi_data)} points returned, '
        f'drawn from {len(rgi_data_sources)} original sources.')
print(f'Columns within the data: {rgi_data.columns}\n\n')

The Specklia query returns all of the original attributes in the RGI shapefile, as well as varying degrees of simplified versions of the original polygon. The simplified polygons have fewer vertices which make for faster querying, if we retrieve only those.

Additionally, source information is supplied for the returned data, and a direct link to the source file.

In [ ]:
display(rgi_data_sources)
display(rgi_data)

We can now plot the RGI v7.0 boundary definition for Austonna ice cap's Basin 3, by extracting it from the returned polygons.

In [ ]:
austfonna_basin_3 = gpd.GeoSeries(
    rgi_data.loc[rgi_data['rgi_id'] == 'RGI2000-v7.0-G-07-01383', 'original_polygons'], crs=4326)

fig, ax = plt.subplots(figsize=(10, 10))

austfonna_basin_3.to_crs(3413).plot(ax=ax, facecolor=geo_filter_colour, edgecolor=geo_filter_colour, alpha=0.5)

ax.set_xlim(982000, 1106000)
ax.set_ylim(-484000, -322000)

ax.set_xlabel('x (meters)')
ax.set_ylabel('y (meters)')
ax.set_title(f"Austfonna Basin 3 ({rgi_data['rgi_id'].values[0]}, EPSG 3413)")
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, crs=3413, attribution=False, zoom=8)

### Querying data
We can now query the two Specklia datasets for our study region.

In [ ]:
cryosat_data = {}
sources = {}
additional_filters = {
    'CryoTEMPO-EOLIS Point Product': [
        {'column': 'uncertainty', 'operator': '<=', 'threshold': 4}],
    'CryoTEMPO-EOLIS Gridded Product': []}

for ds_name, data in eolis_datasets.items():
    query_start_time = perf_counter()
    cryosat_data[ds_name], sources[ds_name] = client.query_dataset(
        dataset_id=data['dataset_id'],
        epsg4326_polygon=austfonna_basin_3.iloc[0],
        min_datetime=datetime(2015, 1, 1),
        max_datetime=datetime(2015, 12, 1),
        columns_to_return=['timestamp', 'elevation', 'uncertainty'],
        additional_filters=additional_filters[ds_name])

    print(f'Query took {perf_counter()-query_start_time:.2f} seconds to complete.')
    print(f'{ds_name} Query complete, {len(cryosat_data[ds_name])} points returned, '
          f'drawn from {len(sources[ds_name])} original sources.')
    print(f'Columns within the data: {cryosat_data[ds_name].columns}\n\n')

Note that within `sources` we have all of the source information from the original product files, including both the Earth Explorer and the NetCDF4 headers, and with the `source_id` and `source_row_id` columns, the ability to trace each point we have just retrieved from its original source file.

If we wanted to, we could run a query that would give us back 100% of the information available in the originally ingested file.

Lastly, we plot the CryoSat-2 data over the top of our study polygon, illustrating the filtering that has been performed:

In [ ]:
for ds_name, cs_data in cryosat_data.items():
    # create a normalised colormap
    norm = mcolors.Normalize(vmin=cs_data['elevation'].min(), vmax=cs_data['elevation'].max())

    ax = austfonna_basin_3.to_crs(epsg=3413).plot(figsize=(10, 10), alpha=0.5)
    cs_data.to_crs(epsg=3413).plot(ax=ax, column='elevation', markersize=.1, norm=norm)
    
    ax.set_xlabel('EPSG 3413 x (m)')
    ax.set_ylabel('EPSG 3413 y (m)')
    ax.set_title(f"{ds_name} Elevation over Austfonna Basin 3 \n(RGI v6.0, EPSG 3413)")
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery, crs=3413, attribution=False, zoom=10)
    cbar = plt.gcf().colorbar(plt.cm.ScalarMappable(norm=norm), ax=ax, shrink=.5)
    cbar.set_label('Elevation (m)')